# Dataclass experiments

In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl'],
      dtype='<U17')

# Code start

In [7]:
# ** IMPORTS
from dataclasses import dataclass, make_dataclass, field
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import pickle

In [15]:
@dataclass
class Scrips:
    symlots: pd.DataFrame = df_symlots
    ohlcs: pd.DataFrame = df_ohlcs
    chains: pd.DataFrame = df_chains
    unds: pd.DataFrame = df_unds
    qopts: pd.Series = qopts

In [16]:
scrips = Scrips()

In [17]:
scrips.unds

,conId,symbol,secType,contract,time,marketDataType,bid,ask,close,low13week,high13week,low26week,high26week,low52week,high52week,avVolume,putOpenInterest,callOpenInterest,histVolatility,iv,undPrice,past12Months,next12Months,nextDate,nextAmount,ticks,margin,comm
0,44652139,CIPLA,STK,"Contract(secType='STK', conId=44652139, symbol...",2020-12-22 00:13:44.509576+00:00,1,-1.00,-1.00,778.50,706.50,829.05,621.05,829.05,351.30,829.05,"8,231,650.00",2443,4789,0.33,0.35,778.50,3.00,2.52,2021-06-29,2.52,NaN,"1,012,967.79",nan
1,44652149,HEROMOTOC,STK,"Contract(secType='STK', conId=44652149, symbol...",2020-12-22 00:13:44.504590+00:00,1,-1.00,-1.00,"2,994.10","2,787.00","3,394.90","2,336.00","3,394.90","1,450.00","3,394.90","1,699,700.00",2753,5858,0.31,0.40,"2,994.10",90.00,75.60,2021-02-16,54.60,NaN,"897,128.98",nan
2,440773528,EICHERMOT,STK,"Contract(secType='STK', conId=440773528, symbo...",2020-12-22 00:13:52.453995+00:00,1,-1.00,-1.00,"2,396.40","2,015.20","2,727.00","1,721.72","2,727.00","1,245.01","2,727.00","1,608,999.00",1535,3459,0.37,0.41,"2,396.40",125.00,nan,NaN,nan,NaN,"841,228.50",nan
3,261024709,JSWSTEEL,STK,"Contract(secType='STK', conId=261024709, symbo...",2020-12-22 00:13:52.452000+00:00,1,-1.00,-1.00,353.50,257.55,376.20,183.65,376.20,130.50,376.20,"8,323,434.00",1189,2064,0.35,0.45,353.50,2.00,1.68,2021-07-01,1.68,NaN,"947,273.43",nan
4,49982069,NTPC,STK,"Contract(secType='STK', conId=49982069, symbol...",2020-12-22 00:14:00.398482+00:00,1,-1.00,-1.00,97.50,78.10,107.25,78.10,108.00,70.05,121.85,"38,337,631.00",2927,6184,0.37,0.35,97.50,3.15,4.45,2021-02-10,2.23,NaN,"552,970.67",nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,44652129,AMBUJACEM,STK,"Contract(secType='STK', conId=44652129, symbol...",2020-12-22 00:13:44.508597+00:00,1,-1.00,-1.00,235.40,180.45,272.00,166.20,272.00,116.55,272.00,"8,188,885.00",1897,5448,0.36,0.38,235.40,20.00,nan,NaN,nan,NaN,"701,691.01",nan
135,49982064,DLF,STK,"Contract(secType='STK', conId=49982064, symbol...",2020-12-22 00:13:52.452998+00:00,1,-1.00,-1.00,214.80,142.50,240.40,133.70,240.40,113.75,263.90,"14,933,325.00",2846,4425,0.53,0.54,214.80,2.00,nan,NaN,nan,NaN,"707,850.00",nan
136,290555779,ICICIGI,STK,"Contract(secType='STK', conId=290555779, symbo...",2020-12-22 00:13:52.453995+00:00,1,-1.00,-1.00,"1,432.70","1,200.00","1,530.00","1,200.00","1,530.00",805.00,"1,530.00","695,640.00",108,339,0.36,0.41,"1,432.70",nan,nan,NaN,nan,NaN,"607,439.76",nan
137,56987139,MFSL,STK,"Contract(secType='STK', conId=56987139, symbol...",2020-12-22 00:14:00.398482+00:00,1,-1.00,-1.00,643.15,566.05,671.70,488.25,671.70,276.35,671.70,"1,679,024.00",541,1437,0.35,0.56,643.15,nan,nan,NaN,nan,NaN,"830,700.00",nan


In [9]:
scrips = Scrips(symlots=df_symlots, ohlcs=df_ohlcs, chains=df_chains, unds=df_unds, qopts=qopts)

In [10]:
fields = [('float_val', float, field(default=np.nan)),
          ('df', pd.DataFrame, field(default_factory=pd.DataFrame)),
          ('int_val', int, field(default_factory=int)),
          ('time_val', datetime, field(default=datetime.now(timezone.utc)))]

In [ ]:
scrips = make_dataclass('')

In [ ]:
Intc = make_dataclass('Intc', fields)
Intc.__module__ = __name__
INTC = Intc()

## Pickle and Unpickle

In [ ]:
import dill
dill.dump(INTC, open(DATAPATH.joinpath('ztest.pkl'), 'wb'))

In [ ]:
x = dill.load(open(DATAPATH.joinpath('ztest.pkl'), 'rb'))

In [ ]:
x.undPrice

In [ ]:
import dill
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from dataclasses import make_dataclass, field

fields = [('float_val', float, field(default=np.nan)),
          ('df', pd.DataFrame, field(default_factory=pd.DataFrame)),
          ('int_val', int, field(default_factory=int)),
          ('time_val', datetime, field(default=datetime.now(timezone.utc)))]

Aclass = make_dataclass('Aclass', fields)

an_instance=Aclass()

dill.dump(an_instance, open('test.pkl', 'wb'))

In [ ]:
an_instance

In [ ]:
import dill
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from dataclasses import make_dataclass, field

fields = [('float_val', float, field(default=np.nan)),
          ('df', pd.DataFrame, field(default_factory=pd.DataFrame)),
          ('int_val', int, field(default_factory=int)),
          ('time_val', datetime, field(default=datetime.now(timezone.utc)))]

Aclass = make_dataclass('Aclass', fields)
Aclass.__module__ = __name__

an_instance=Aclass()

dill.dump(an_instance, open('test1.pkl', 'wb'))

In [ ]:
an_instance.__dict__

In [ ]:
dill.load(open('test1.pkl', 'rb'))

In [ ]:
an_instance

In [ ]:
fields = [('df1', pd.DataFrame, field(default_factory=pd.DataFrame)),
          ('df2', pd.DataFrame, field(default_factory=pd.DataFrame))]


Dclass = make_dataclass('Dclass', fields)
Dclass.__module__ = __name__

an_instance=Dclass()

dill.dump(an_instance, open('test2.pkl', 'wb'))

In [ ]:
dill.load(open('test2.pkl', 'rb'))